In [1]:
import requests as rsqt
from bs4 import BeautifulSoup
import json

url = 'https://www.udemy.com'
res = rsqt.get(url)
soup = BeautifulSoup(res.text,'html.parser')

catg_menu = soup.find_all('a', {'class': 'js-side-nav-cat'})
final_data = []
frms = []
frms_hrefs = []
sous_frms = []
sous_frms_hrefs = []
frms_dom_part = []

# recuperer les formations et sous-fromations et leur hrefs(lien) dans des listes separées 
for index,item in enumerate(catg_menu):
    str = item.get('href').split('/')
    if(len(str) == 4):
        frms.append(catg_menu[index].text)
        frms_hrefs.append(catg_menu[index].get('href'))
        frms_dom_part.append(item.get('href').split('/')[2])
    elif (len(str) == 5):
        sous_frms.append(catg_menu[index].text)
        sous_frms_hrefs.append(catg_menu[index].get('href'))

# reformuler les données des listes afin d'avoir un objet organisée 
# chaque formations et ses sous formations et leur hrefs
for index,item in enumerate(frms):
    dict = {}
    dict['formation'] = item
    dict['formation_href'] = frms_hrefs[index]
    dict['sous_formations'] = []
    final_data.append(dict)
    for index_s_f,s_item in enumerate(sous_frms):
        s_f_dict = {}
        s_f_dict['sous_formation'] = s_item
        str_= sous_frms_hrefs[index_s_f].split('/')[2]
        index_str = frms_dom_part.index(str_)
        if(index == index_str):
            s_f_dict['sous_formation_href'] = sous_frms_hrefs[index_s_f]
            final_data[index]['sous_formations'].append(s_f_dict)
            for ind in range(len(final_data[index]['sous_formations'])):
                final_data[index]['sous_formations'][ind]['sujets'] = []
# stocker la data dans un fichier json pour les lires après pour scrapper les sujets (sous sous-formations)
# cette etape optimise les requetes vers le site      
with open('data.json', 'w') as fichier_sortie:
    json.dump(final_data, fichier_sortie, indent=4) 


In [33]:
index_sous_form = 0

In [35]:
index_sous_form +=1
print(index_sous_form)

1


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from unidecode import unidecode
import json

# parcourir chaque sous formations et formations poue extraire les sujets et 
# les informations tel que le prix, la durée ...
max_pages = 1
with open('./data.json' ,'r') as file:
    frms_js = json.load(file)
    nbr_frms = len(frms_js)
    #boucler les sur les pages
    for nb_p in [1]:
        driver = webdriver.Edge()
        # boucler sur les les formations
        for i in range(nbr_frms):
            s_fr = frms_js[i]['sous_formations']
            nbr_sous_frms = len(s_fr)
            # la première sous formation existe dans le dev qui est dev web
            for j in range(nbr_sous_frms):
                driver = webdriver.Edge()
                url_fr = url + s_fr[j]['sous_formation_href']+f'?p={nb_p}'
                print(url_fr)
                driver.get(url_fr)
                wait = WebDriverWait(driver,30)
                element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.course-list--container--FuG0T")))
                element_source = element.get_attribute("outerHTML")
                soup = BeautifulSoup(element_source, "html.parser")
                page_sujets = soup.find_all('div',{'class' : 'course-card-module--main-content--3Uvsz course-card-module--has-price-text--3EF7y'})            
                for elem_sj in page_sujets:
                    dict_sujet = {}
                    try:
                        dict_sujet['sujet_name'] = unidecode(elem_sj.find('a').text)
                        dict_sujet['sujet_desc'] = unidecode(elem_sj.find('span',{'data-testid' : 'seo-headline'}).text)
                        dict_sujet['sujet_rating'] = unidecode(elem_sj.find('span',{'data-testid' : 'seo-rating'}).text)
                        dict_sujet['sujet_reviews'] = unidecode(elem_sj.find('span',{'data-testid' : 'seo-num-reviews'}).text)
                        dict_sujet['sujet_duree'] = unidecode(elem_sj.find('span',{'data-testid' : 'seo-content-info'}).text)
                        dict_sujet['sujet_nbr_lecturs'] = unidecode(elem_sj.find('span',{'data-testid' : 'seo-num-lectures'}).text)
                        dict_sujet['sujet_level'] = unidecode(elem_sj.find('span',{'data-testid' : 'seo-instructional-level'}).text)
                        dict_sujet['sujet_price'] = '9,99 E'
                        #dict_sujet['sujet_price'] = unidecode(elem_sj.find('span',{'data-testid' : 'seo-current-price'}).text)
                    except:
                        pass
                    s_fr[j]['sujets'].append(dict_sujet) 
                driver.quit()            
    with open('data_sj.json', 'w') as fichier_sortie:
        json.dump(frms_js, fichier_sortie, indent=4) 
        

In [57]:
import json
# code pour extraire les formations
res = {}
data = []
with open('./data_sj.json' ,'r') as file:
    data = json.load(file)
    for i in range(len(data)):
        res['udemy-form' + str(i)] = data[i]['formation']
test = []
with open('requete.sql', 'w') as file:
    #extraire les requetes d'insertion des domains
    for key, value in res.items():
        file.write(f"INSERT INTO DomaineFormation (RefDomaine, DomaineDeFormation) VALUES ('{key}', '{value}');\n")
    file.write(f"----------------------------------------------------------------------------------------\n\n")
    #extraire les requetes d'insertion des sousDomains
    counter = 0
    for i in range(len(data)):
        formation = data[i]['formation']
        for j in range(len(data[i]['sous_formations'])):
            s_formation = data[i]['sous_formations'][j]['sous_formation']
            link_s_formation = data[i]['sous_formations'][j]['sous_formation_href']
            unique_key = 'udemy_sfor_'+str(counter)
            file.write(f"INSERT INTO SousDomaineFormation (RefSousDomaine,SousDomaineDeFormation,RefDomaine)VALUES ('{unique_key}','{s_formation}', (SELECT RefDomaine FROM DomaineFormation WHERE DomaineDeFormation = '{formation}'));\n")
            counter+=1
    
    file.write(f"----------------------------------------------------------------------------------------\n\n")
    
    #extraire les requetes d'insertion des SousSousDomains
    index = 10
    counter = 0
    counter_2 = 0
    lis_item = []
    for i in range(len(data)):
        formation = data[i]['formation']
        for j in range(len(data[i]['sous_formations'])):
            s_formation = data[i]['sous_formations'][j]['sous_formation']
            link_s_formation = data[i]['sous_formations'][j]['sous_formation_href']
            unique_key_sf = 'udemy_sfor_'+str(counter)
            counter+=1
            for k in range(len(data[i]['sous_formations'][j]['sujets'])):
                sousSformation = data[i]['sous_formations'][j]['sujets'][k]
                unique_key_sous_sf = 'udemy_sous_sfor_'+str(counter_2)
                file.write(f"""INSERT INTO SOUSSOUSDomaineFormation (RefSOUSSOUSDomaineF,RefSOUSDomaineF,Le_nom,Descriptio,Notes,Nombre_avis,Duree,Organisation)
                        VALUES(
                           '{unique_key_sous_sf}',
                           (SELECT RefSousDomaine FROM SousDomaineFormation WHERE SousDomaineDeFormation = '{s_formation}'),
                           '{sousSformation['sujet_name'].replace("'", "''")}',
                           '{sousSformation['sujet_desc'].replace("'", "''")}',
                           '{sousSformation['sujet_rating']}',
                           '{sousSformation['sujet_reviews']}',        
                            '{sousSformation['sujet_duree']}',
                            'Udemy'
                    );\n""")
                counter_2+=1